# 🎼🤖 ArmonIA - generación de musica con redes neuronales

## Introducción

armonIA es un proyecto que busca generar música utilizando redes neuronales. El objetivo es crear un modelo capaz de componer música original en diferentes estilos de musica clasica, utilizando técnicas de aprendizaje profundo y procesamiento de lenguaje natural. En este proyecto, se utilizará Torch para construir y entrenar el modelo.


### Prologo

Antes de comenzar con el desarrollo del proyecto, es importante entender los datos que se tienen y lo que cada uno representa, a continuación se lista cada uno de los datos de entrada 

Características de entrada 

| Característica | Tipo de Variable | Descripción                                                                |
|----------------|------------------|----------------------------------------------------------------------------|
| Pitch          | Categórica       | Representa la altura de la nota musical (`note.pitch`).                    |
| Step           | Numérica         | Diferencia entre el `start` de la nota actual y el `start` de la anterior. |
| Duration       | Numérica         | Diferencia entre el `end` y el `start` de la nota actual.                  |
| Velocity       | Numérica         | Representa la velocidad de la nota (`note.velocity`).                      |


Como las variables Step y Duration son numéricas, se normalizan para que estén en el rango [0, 1]. La variable Velocity se normaliza para que esté en el rango [0, 127]. La variable Pitch se convierte a una representación numérica utilizando un diccionario de mapeo.


en este caso no es necesario eliminar los datos duplicados, ya que cada nota es única y no se repite en el tiempo. Sin embargo, es importante asegurarse de qe no haya datos nulos o vacíos en el conjunto de datos.

Se realiza un diseño primero de la arquitectura del modelo, para luego proceder a la implementación del mismo. El modelo se basa en una red neuronal LSTM (Long Short-Term Memory) que es capaz de aprender patrones en secuencias de datos. La arquitectura del modelo se puede ver en la siguiente imagen: 

<div style="display: flex; gap: 32px; justify-content: center; item-align: center; width: 100%;">
    <!-- <img style="width:400px" src="https://www.researchgate.net/publication/385782855/figure/fig1/AS:11431281290276122@1731539282950/LSTM-model-architecture-for-song-generation-Adapted-from-9.ppm" /> -->
    <img src="./lstm.png" style="width:500px" alt="LSTM model architecture for song generation" />
    <div>
        Se utilizará un modelo LSTM para la generación de música debido a su capacidad para manejar secuencias de datos. El modelo tomará como entrada una secuencia de notas y acordes, y generará una nueva secuencia de notas y acordes como salida.
    </div>
</div>

## Fases del proyecto

### 1. Pre procesamiento de datos

Para entrenar un modelo de generación de música, es necesario contar con un conjunto de datos que contenga ejemplos de música en el estilo deseado. En este caso, se utilizará un conjunto de datos de partituras musicales en formato MIDI. Se utilizarán bibliotecas como `pretty-midi` para procesar y analizar los archivos MIDI.

En esta etapa se realizarán las siguientes tareas:

1. Cargar los archivos MIDI y extraer las notas y acordes.
2. Convertir las notas y acordes en una representación numérica que pueda ser utilizada por el modelo.
3. Dividir los datos en secuencias de longitud fija para facilitar el entrenamiento del modelo.
4. Normalizar los datos para mejorar la convergencia del modelo.
5. Dividir los datos en conjuntos de entrenamiento y validación.



### 2. Construcción, entrenamiento y evaluación del modelo


Para la construcción del modelo se utilizará la biblioteca `torch` para crear una red neuronal LSTM. El modelo tomará como entrada una secuencia de notas y acordes, y generará una nueva secuencia de notas y acordes como salida. Se utilizarán capas LSTM para capturar las dependencias temporales en los datos, y se aplicarán técnicas de regularización como Dropout para evitar el sobreajuste.

Lista de pasos a seguir:

1. Definir la arquitectura del modelo LSTM.
2. Definir la función de pérdida y el optimizador.
3. Implementar el bucle de entrenamiento y validación.
4. Guardar el modelo entrenado para su uso posterior.
5. Evaluar el modelo utilizando métricas como la pérdida y la precisión.


### 3. Generación de música

Para generar música, se utilizará el modelo entrenado para predecir la siguiente nota o acorde dado una secuencia de notas y acordes inicial. Se implementará un algoritmo de muestreo para generar nuevas secuencias de notas y acordes, y se utilizarán técnicas de post-procesamiento para convertir las secuencias generadas en archivos MIDI.

Lista de pasos a seguir:

1. Cargar el modelo entrenado.
2. Definir una secuencia inicial de notas y acordes.
3. Utilizar el modelo para predecir la siguiente nota o acorde.
4. Repetir el proceso para generar una secuencia completa de notas y acordes.
5. Convertir la secuencia generada en un archivo MIDI.

---

## 1. Pre procesamiento de datos

### 1.1 Impotar librerias necesarias

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn 

from typing import List, Tuple, Dict, Any, Union 


from torch.utils.data import Dataset, DataLoader
import pretty_midi
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

# 1.2 Crear funciones para cargar y procesar los archivos MIDI

se crean las funciones necesarias para cargar y procesar los archivos MIDI. Se utiliza la biblioteca `pretty_midi` para cargar los archivos MIDI y extraer las notas y acordes. Se definen funciones para convertir las notas y acordes en una representación numérica, dividir los datos en secuencias de longitud fija, normalizar los datos y dividir los datos en conjuntos de entrenamiento y validación.




In [ ]:
class MusicDataset(Dataset):
    def __init__(self, X: List[np.ndarray], y: List[int], seq_length: int = 32, label_encoder: LabelEncoder):

        self.X = X
        self.y = y
        self.label_encoder = 

        self.data = data
        self.seq_length = seq_length
        self.scaler = MinMaxScaler()
        self.label_encoder = LabelEncoder()
        self._prepare_data()

    def __len__(self):
        return len(self.X)

    def _prepare_data(self):
        # Extract features and labels
        features, labels = zip(*self.data)
        
        # Scale features
        features = np.array(features)
        self.scaler.fit(features.reshape(-1, features.shape[-1]))
        features = self.scaler.transform(features.reshape(-1, features.shape[-1])).reshape(features.shape)
        
        # Encode labels
        labels = np.array(labels)
        self.label_encoder.fit(labels)
        labels = self.label_encoder.transform(labels)
        
        # Split into sequences
        self.features = []
        self.labels = []
        
        for i in range(len(features)):
            for j in range(0, len(features[i]) - self.seq_length + 1):
                self.features.append(features[i][j:j + self.seq_length])
                self.labels.append(labels[i])
        
        self.features = np.array(self.features)
        self.labels = np.array(self.labels)

In [ ]:
class Preprocessor:
    """permite cargar y preprocesar los archivos midi."""

    def __init__(self):

        self.scaler = MinMaxScaler()
        self.label_encoder = LabelEncoder()


    # *~~~ api start ~~~*
    def load_midi_files(self, filepath: str, target_program = None) -> List[pretty_midi.Note]:
        """
        Carga los archivos MIDI y extrae notas del instrumento especificado.
        """
        try: 
            midi_data = pretty_midi.PrettyMIDI(filepath)
            # Extraer notas de un instrumento específico (ej. piano)

            if target_program is not None:
                instruments = [inst for inst in midi_data.instruments if inst.program == target_program]
                if not instruments:
                    return []
                
                instrument = instruments[0]
            else:
                instrument = midi_data.instruments[0]

            # print(f"list of instruments: {midi_data.instruments}")

            notes = instrument.notes
            return notes
        
        except Exception as e:
            print(f"❌ Error al cargar archivo midi {e}")
            return None

    def extract_features(self, notes: pretty_midi.PrettyMIDI) -> np.ndarray:
        """
        Extrae características de cada nota: pitch, step, duration (y opcionalmente velocity).
        """

        # se ordenan las notas por tiempo de inicio para
        # garantizar que las notas tengan el mismo orden de la melodía
        notes = sorted(notes, key=lambda x: x.start) # x[1] es start
    
        data = []
        for i in range(1, len(notes)):
            note = notes[i]
            prev_note = notes[i-1]
            pitch = note.pitch
            pitch_name = pretty_midi.note_number_to_name(pitch)
            step = note.start - prev_note.start
            duration = note.end - note.start
            velocity = note.velocity
            data.append([pitch_name, step, duration, velocity])

        return np.array(data, dtype=object)

    def get_notes_dataframe(self, data: np.ndarray) -> pd.DataFrame:
        columns = ["pitch", "step", "duration", "velocity"]
        df = pd.DataFrame(data, columns=columns)

        # Convertir pitch a string explícitamente (por si NumPy lo convirtió)
        df["pitch"] = df["pitch"].astype(str)

        return df

   
    def build_sequences(self, data: np.ndarray, context_size: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Construye pares (X, Y) para entrenamiento, a partir del array de notas completo.
    
        - `X`: secuencia de `context_size` notas [nota_t−c, ..., nota_t−1]
        - `Y`: nota objetivo nota_t
        """
        X, y = [], []

        for i in range(context_size, len(data)):
            context = data[i-context_size: i+1]


            X.append(context[:-1])  # Todas las notas menos la última
            y.append(context[-1])  # Solo la última nota

        X = np.array(X, dtype=object)
        y = np.array(y, dtype=object)

        return X, y

    def normalize_features(self, X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Aplica normalización 

        """
            
        # Convertimos los arrays a float para evitar errores
        X = X.copy()
        y = y.copy()


    def plot_data(self):
        """Graficar los datos."""
        pass

    # *~~~ api end ~~~*

    # *~~~ internal fn start ~~~*
    # *~~~ internal fn end ~~~*
    

      


In [ ]:
# listar todos los archivos midi 

_path = "dataset/music_artist/haydn"


data = []

preprocessor = Preprocessor()



for filename in os.listdir(_path):
    file_path = os.path.join(_path, filename)

    notes = preprocessor.load_midi_files(file_path)

    if notes:
        features = preprocessor.extract_features(notes)

        # añadir nota de inicio
        start_note = np.array([["START", 0.0, 0.0, 0]], dtype=object)

        # añadir nota de fin
        end_note = np.array([["END", 0.0, 0.0, 0]], dtype=object)

        # Unir todo
        song_sequence = np.vstack([start_note, features, end_note])

        data.extend(song_sequence.tolist())

    all_sequence = preprocessor.build_sequences(data, context_size=10)




# df = preprocessor.get_notes_dataframe(data)

# print(df)

# print(preprocessor.get_notes_dataframe(all_features[0]))


ValueError: Buffer has wrong number of dimensions (expected 1, got 3)

In [ ]:
# BORRAR ESTA CELDA

preprocessor = Preprocessor()
notes = preprocessor.load_midi_files('dataset/example.midi')
data = preprocessor.extract_features(notes)

print("notes:", data)

df = pd.DataFrame(data, columns=['pitch', 'step', 'duration', 'velocity'])

df

In [ ]:
# conver to pandas for visualization

# test with example.mid
def convert_to_dataframe(data):
    df = pd.DataFrame(data, columns=['pitch', 'step', 'duration', 'velocity'])
    return df

def plot_data(df):
    plt.figure(figsize=(10, 6))
    plt.scatter(df['step'], df['duration'], c=df['pitch'], cmap='viridis', alpha=0.5)
    plt.colorbar(label='Pitch')
    plt.xlabel('Step')
    plt.ylabel('Duration')
    plt.title('MIDI Note Data')
    plt.show()



In [57]:
_path = 'dataset/music_artist/haydn'

# Extraer notas del archivo MIDI

data = load_all_notes_from_folder(_path)

len(data)

# Convertir a DataFrame para visualización
df = convert_to_dataframe(data)

print(df.head())


   pitch      step  duration  velocity
0     67  0.335195  0.552632        49
1     71  0.552632  0.078947        47
2     69  0.078947  0.322580        47
3     62  0.322580  0.312500        49
4     64  0.312500  0.312500        45


In [61]:
def create_sequences(df, context=10):
    data = df[['pitch', 'step', 'duration', 'velocity']].values
    sequences = []
    targets = []
    for i in range(len(data) - context):
        seq = data[i:i+context]
        target = data[i+context]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

X, y = create_sequences(df, context=10)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

### 3. Construcción del modelo


Características de entrada 

- Pitch (altura de la nota): variable categ´orica que representa la nota musical.
- Step (tiempo): variable numerica que se debe calcular como el start de la nota actual menos el start de la nota anterior, y representa el espacio de tiempo que hay entre la   ota anterior y la nota actual.
- Duration (duración): variable numérica, se calcula como la diferencia entre el tiempo en el que termina la nota actual (end) menos el tiempo en el que inicio la nota (start), o que refleja la duracion en tiempo de la nota.
- velocity (velocidad): variable numérica que representa la velocidad de la nota


Para construir el modelo de generación de música, se utilizará una red neuronal recurrente (RNN) con capas LSTM. Las RNN son adecuadas para tareas de secuencias, como la generación de música, ya que pueden capturar dependencias a largo plazo en los datos.
Se utilizará la biblioteca `torch` para construir el modelo. A continuación se presenta un ejemplo de cómo se puede definir una clase para el modelo:

In [62]:
class NoteDataset(Dataset):
    def __init__(self, sequences: list, targets: list):
        """
        Args:
        sequennces: lista de notas que serán usadas como entrada del modelo (contexto de notas anteriores)
        targets: lista de notas que serán usadas como salida del modelo (notas a predecir)

        x = [Nota_{t-c}, ..., Nota_{t-1}]
        y = [Nota_{t}]
        """
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        """Número de secuencias en el dataset."""
        return len(self.sequences)
    
    def __getitem__(self, idx):
        """
        Obtiene un elemento del dataset dado un índice.

        convierte la secuencia y el target a tensores de PyTorch.
        """
        return torch.tensor(self.sequences[idx], dtype=torch.float32), \
                torch.tensor(self.targets[idx], dtype=torch.float32)

In [63]:
class MusicLSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int):
        """
        Initialize the LSTM model.

        Args:
            input_size (int): Size of the input features.
            hidden_size (int): Number of features in the hidden state.
            num_layers (int): Number of recurrent layers.

        
        """
        super(MusicLSTM, self).__init__()
        
        # Definimos la arquitectura de la red LSTM

        # input_size: número de características de entrada (4 en este caso: pitch, step, duration, velocity)
        # hidden_size: número de características en el estado oculto
        # num_layers: número de capas LSTM apiladas
        # batch_first=True: la entrada y salida de la LSTM serán de tamaño (batch, seq, feature)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)


        # Definimos las capas de salida para cada una de las características a predecir
        # pitch, step, duration y velocity
        # Cada una de estas capas toma como entrada el estado oculto final de la LSTM y produce una salida 
        self.fc_pitch = nn.Linear(hidden_size, 128)
        self.fc_step = nn.Linear(hidden_size, 1)
        self.fc_duration = nn.Linear(hidden_size, 1)
        self.fc_velocity = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Forward pass de la red LSTM.
        Args:
            x: tensor de entrada con forma (batch_size, seq_length, input_size)
        Returns:
            pitch: tensor de salida para la predicción de pitch
            step: tensor de salida para la predicción de step
            duration: tensor de salida para la predicción de duration
            velocity: tensor de salida para la predicción de velocity
        """
        out,_ = self.lstm(x)
        out = out[:, -1, :] # tomamos solo el último paso
        pitch = self.fc_pitch(out)
        step = self.fc_step(out)
        duration = self.fc_duration(out)
        velocity = self.fc_velocity(out)
        return pitch, step, duration, velocity


In [64]:
from tqdm import tqdm

def train_model(model: nn.Module, dataloader: DataLoader, epochs: int, criterion_pitch: nn.Module, criterion_reg: nn.Module, optimizer: torch.optim.Optimizer):
    """
    Train the LSTM model.
    Args:
        model (nn.Module): The LSTM model to train.
        dataloader (DataLoader): DataLoader for the training data.
        epochs (int): Number of epochs to train.
        criterion_pitch (nn.Module): Loss function for pitch.
        criterion_reg (nn.Module): Loss function for regression targets.
        optimizer (torch.optim.Optimizer): Optimizer for training.
    """
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for batch_x, batch_y in loop:

            # x_batch: (batch_size, seq_length, input_size)
            # y_batch: (batch_size, 4) (pitch, step, duration, velocity)

            # separar cada una de las características de salida
            pitch_true = batch_y[:, 0].long()
            step_true = batch_y[:, 1].unsqueeze(1)
            duration_true = batch_y[:, 2].unsqueeze(1)
            velocity_true = batch_y[:, 3].unsqueeze(1)

            # limitar el gradiente
            optimizer.zero_grad()

            # forward pass
            pitch_pred, step_pred, duration_pred, velocity_pred = model(batch_x)


            # calcular la pérdida para cada una de las características
            loss_pitch = criterion_pitch(pitch_pred, pitch_true)
            loss_step = criterion_reg(step_pred, step_true)
            loss_duration = criterion_reg(duration_pred, duration_true)
            loss_velocity = criterion_reg(velocity_pred, velocity_true)

            loss = loss_pitch + loss_step + loss_duration + loss_velocity

            # backward pass y optimización
            loss.backward()
            optimizer.step()

            # Agregar la pérdida total
            total_loss += loss.item()

        # ímprimir la pérdida total por cada epoch 
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [77]:
# Hiperparámetros
input_size = 4           # pitch, step, duration, velocity
hidden_size = 128
num_layers = 8
batch_size = 64
epochs = 100
learning_rate = 0.001

# Dataset y DataLoader
train_dataset = NoteDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Inicializar modelo
model = MusicLSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

# Funciones de pérdida
criterion_pitch = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

# Optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Llamar al entrenamiento
train_model(
    model=model,
    dataloader=train_loader,
    criterion_pitch=criterion_pitch,
    criterion_reg=criterion_reg,
    optimizer=optimizer,
    epochs=epochs
)


Epoch 1, Loss: 363801.3507


Epoch 2, Loss: 93435.4376


Epoch 3, Loss: 36339.0696


Epoch 4, Loss: 29205.0231


Epoch 5, Loss: 28836.7888


...

Epoch 96, Loss: 4764.7320


Epoch 97, Loss: 4632.0488


Epoch 98, Loss: 4465.4489


Epoch 99, Loss: 4022.3436


Epoch 100, Loss: 4107.3184


In [78]:
def generate_sequence(model, seed_sequence, length, context=10, device='cpu'):
    model.eval()
    generated = []
    current_seq = seed_sequence.copy()

    for _ in range(length):
        # Asegura que solo se tomen las últimas `context` notas
        input_seq = current_seq[-context:]
        input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            pitch_logits, step, duration, velocity = model(input_tensor)

        # Predecir pitch con argmax
        pitch = torch.argmax(pitch_logits, dim=1).item()

        # Extraer valores escalares de salida
        step = step.item()
        duration = duration.item()
        velocity = velocity.item()

        # Guardar la nota generada
        generated.append([pitch, step, duration, velocity])

        # Agregar la nueva nota a la secuencia
        current_seq.append([pitch, step, duration, velocity])

    return generated


In [79]:
def save_midi(notes, output_path, instrument_name="Acoustic Grand Piano"):
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program(instrument_name))
    start = 0
    for note in notes:
        pitch, step, duration, velocity = note
        start += step
        end = start + duration
        note_obj = pretty_midi.Note(velocity=int(velocity), pitch=int(pitch), start=start, end=end)
        instrument.notes.append(note_obj)
    pm.instruments.append(instrument)
    pm.write(output_path)

In [80]:
# Parámetros
context = 10
num_notes = 200
device = 'cpu'  # o 'cuda' si estás usando GPU

# Semilla: las primeras 10 notas del set de entrenamiento
seed_sequence = X_train[0].tolist()

# Generar notas
generated_notes = generate_sequence(
    model=model,
    seed_sequence=seed_sequence,
    length=num_notes,
    context=context,
    device=device
)


In [ ]:
generated_notes

In [76]:
save_midi(
    notes=generated_notes,
    output_path='generated_music.mid',
    instrument_name="Acoustic Grand Piano"
)